## Prev Created Code

In [1]:
#r "nuget: Castle.Core"

using System.Collections.Immutable;
using System.Linq.Expressions;
using System.Reflection;
using Castle.DynamicProxy;

public class Executed
{
  private readonly Func<int, bool> _validate;
  private readonly string _message;

  public Executed(Func<int, bool> validate, string message)
  {
    _validate = validate;
    _message = message;
  }

  public void Verify(int count)
  {
    if (!_validate.Invoke(count))
    {
      throw new Exception($"Executed wrong number of times. Expected: {_message} Executed: {count}");
    }
  }

  public static Executed Once => new (x => x == 1, "Once");
  public static Executed Never => new (x => x == 0, "Never");
  public static Executed AtLeast(int times) => new (x => x >= times, $"At least {times}");
  public static Executed AtMost(int times) => new (x => x <= times, $"At most {times}");
}

public class Arg
{
  public static T IsAny<T>() => throw new InvalidOperationException();
  public static T Is<T>(Func<T, bool> matcher) => throw new InvalidOperationException();
}

public class MockInvocation 
{
  public MockInvocation(IInvocation invocation)
  {
    Arguments = invocation.Arguments.ToImmutableArray();
    Method = invocation.Method;
  }
  
  public MethodInfo Method { get; }
  public ImmutableArray<object> Arguments { get; }
}

public class MockInterceptor : StandardInterceptor
{
  private readonly List<MockInvocation> _invocations = new();

  public IReadOnlyList<MockInvocation> Invocations => _invocations;

  protected override void PreProceed(IInvocation invocation)
  {
    _invocations.Add(new MockInvocation(invocation));
    base.PreProceed(invocation);
  }

  protected override void PerformProceed(IInvocation invocation)
  {
  }
}

public class ProxyFactory
{
  private readonly ProxyGenerator _generator = new ();

  public T GetProxy<T>(IInterceptor interceptor) where T:class => _generator.CreateInterfaceProxyWithoutTarget<T>(interceptor);
}

public static class FactoryProvider 
{
  public static ProxyFactory Factory {get;} = new ProxyFactory();
}

public class Visitor : ExpressionVisitor
{
  private static readonly MethodInfo IsMethod = typeof(Arg).GetMethods(BindingFlags.Static | BindingFlags.Public).Single(x => x.Name == "Is");
  public MethodInfo Method { get; private set; }
  public IReadOnlyList<Func<object, bool>> ArgumentValidators { get; private set; }
  protected override Expression VisitMethodCall(MethodCallExpression node)
  {
    Method = node.Method;
    var argumentValidators = new List<Func<object, bool>>();
    foreach (var argument in node.Arguments)
    {
      if (argument.NodeType == ExpressionType.Constant)
      {
        argumentValidators.Add(x => x.Equals(((ConstantExpression)argument).Value));
      }
      else if (argument.NodeType == ExpressionType.Call)
      {
        VisitItExpression(argumentValidators, (MethodCallExpression)argument);
      }
    }

    ArgumentValidators = argumentValidators;
    return node;
  }
  
  private void VisitItExpression(List<Func<object,bool>> argumentValidators, MethodCallExpression argument)
  {
    if (argument.Method.DeclaringType != typeof(Arg))
    {
      throw new InvalidOperationException("Only Arg.Is* is supported");
    }
    else if (argument.Method.IsGenericMethod)
    {
      var genericMethodDefinition = argument.Method.GetGenericMethodDefinition();
      if (genericMethodDefinition == IsMethod)
      {
        argumentValidators.Add(VisitIsMethod(argument));
      }
      // Handle Other Methods...
    }
  }

  private Func<object, bool> VisitIsMethod(MethodCallExpression argument)
  {
    var arg = (LambdaExpression)argument.Arguments.Single(); // We know that Arg.Is has only one argument
    return BuildArgumentValidator(arg.Parameters.Single().Type, arg);
  }

  private Func<object, bool> BuildArgumentValidator(Type castToType, Expression validateExpression)
  {
    var arg = Expression.Parameter(typeof(object));
    var casted = Expression.Convert(arg, castToType);
    var body = Expression.Invoke(validateExpression, casted);
    return Expression.Lambda<Func<object, bool>>(body, arg).Compile();
  }
}

public class MockInvocationMatch
{
  private readonly MethodInfo _method;
  private readonly IReadOnlyList<Func<object, bool>> _argumentValidators;

  public MockInvocationMatch(Expression expression)
  {
    var visitor = new Visitor();
    visitor.Visit(expression);
    (_method, _argumentValidators) = (visitor.Method, visitor.ArgumentValidators);
  }
  public bool IsMatch(MockInvocation invocation) => invocation.Method == _method && invocation.Arguments.Zip(_argumentValidators, (x, y) => y(x)).All(x => x);
}


public class MockObject<T>  where T : class 
{
  private readonly Lazy<T> _lazy;
  private readonly MockInterceptor _interceptor = new();
  
  public MockObject() 
  {
    _lazy = new Lazy<T>(CreateInstance);
  }

  public void Verify(Expression<Action<T>> action, Executed times)
  {
    var invocationMatch = new MockInvocationMatch(action);
    times.Verify(_interceptor.Invocations.Count(x => invocationMatch.IsMatch(x)));
  }

  public T Object => _lazy.Value;

  private T CreateInstance() => FactoryProvider.Factory.GetProxy<T>(_interceptor);
}

The below script needs to be able to find the current output cell; this is an easy method to get it.

Installed Packages Castle.Core, 5.1.1

## New code

``` CSharp
public class MockObject<T>  where T : class 
{
  private readonly MockInterceptor _interceptor = new();
  // ...

  public SetupVoidMethodBuilder<T> Setup(Expression<Action<T>> action) => new (_interceptor, action);
  public SetupFuncMethodBuilder<T, TResult> Setup<TResult>(Expression<Func<T, TResult>> action) => new (_interceptor, action);
}

```CSharp
public class MockInterceptor : StandardInterceptor
{
    public Action<IInvocation>? Perform { get; set; }

    protected override void PerformProceed(IInvocation invocation) => Perform?.Invoke(invocation);
    // ...
}

In [2]:
public class MockInterceptor : StandardInterceptor
{
  private readonly List<MockInvocation> _invocations = new();

  public IReadOnlyList<MockInvocation> Invocations => _invocations;

  public Action<IInvocation>? Perform { get; set; }

  protected override void PreProceed(IInvocation invocation)
  {
    _invocations.Add(new MockInvocation(invocation));
    base.PreProceed(invocation);
  }

  protected override void PerformProceed(IInvocation invocation) => Perform?.Invoke(invocation);
}


(7,29): warning CS8632: The annotation for nullable reference types should only be used in code within a '#nullable' annotations context.



In [3]:
public class SetupVoidMethodBuilder<T>
{
  private readonly MockInterceptor _interceptor;
  private Exception? _expressionToThrow;

  protected SetupVoidMethodBuilder(MockInterceptor interceptor, MockInvocationMatch invocationMatch)
  {
    _interceptor = interceptor;
    _interceptor.Perform += invocation =>
    {
      if (invocationMatch.IsMatch(new MockInvocation(invocation)))
      {
        Execute(invocation);
      }
    };
  }
  protected virtual void Execute(IInvocation invocation)
  {
    if (_expressionToThrow != null)
      throw _expressionToThrow;
  }

  public SetupVoidMethodBuilder(MockInterceptor interceptor, Expression<Action<T>> action)
    : this(interceptor, new MockInvocationMatch(action))
  {
  }

  public void Throws(Exception exception) => _expressionToThrow = exception;
}


(4,20): warning CS8632: The annotation for nullable reference types should only be used in code within a '#nullable' annotations context.



In [4]:
public class SetupFuncMethodBuilder<T, TResult> : SetupVoidMethodBuilder<T>
{
  private TResult? _result;

  public SetupFuncMethodBuilder(MockInterceptor interceptor, Expression<Func<T, TResult>> action)
    : base(interceptor, new MockInvocationMatch(action))
  {
  }

  public void Returns(TResult value)
  {
    _result = value;
  }

  protected override void Execute(IInvocation invocation)
  {
    base.Execute(invocation);
    if (_result != null)
    {
      invocation.ReturnValue = _result;
    }
  }
}


(3,18): warning CS8632: The annotation for nullable reference types should only be used in code within a '#nullable' annotations context.



In [5]:
public class MockObject<T> where T : class
{
  private readonly Lazy<T> _lazy;
  private readonly MockInterceptor _interceptor = new();

  public MockObject()
  {
    _lazy = new Lazy<T>(CreateInstance);
  }

  public SetupVoidMethodBuilder<T> Setup(Expression<Action<T>> action) => new(_interceptor, action);
  public SetupFuncMethodBuilder<T, TResult> Setup<TResult>(Expression<Func<T, TResult>> action) => new(_interceptor, action);

  public void Verify(Expression<Action<T>> action, Executed times)
  {
    var invocationMatch = new MockInvocationMatch(action);
    times.Verify(_interceptor.Invocations.Count(x => invocationMatch.IsMatch(x)));
  }

  public T Object => _lazy.Value;

  private T CreateInstance() => FactoryProvider.Factory.GetProxy<T>(_interceptor);
}

## Demo

In [6]:
interface IFoo2
{
    int M(string arg);
}

var mockObj = new MockObject<IFoo2>();

In [7]:
mockObj.Setup(x => x.M(Arg.Is<string>(arg => arg == "Hello"))).Returns(10);
mockObj.Setup(x => x.M(Arg.Is<string>(arg => arg == "World"))).Throws(new InvalidOperationException());

In [8]:
mockObj.Object.M("Hello")

10

In [9]:
mockObj.Object.M("World")

Error: System.InvalidOperationException: Operation is not valid due to the current state of the object.
   at Submission#5.SetupVoidMethodBuilder`1.Execute(IInvocation invocation)
   at Submission#6.SetupFuncMethodBuilder`2.Execute(IInvocation invocation)
   at Submission#5.SetupVoidMethodBuilder`1.<>c__DisplayClass2_0.<.ctor>b__0(IInvocation invocation)
   at Submission#4.MockInterceptor.PerformProceed(IInvocation invocation)
   at Castle.DynamicProxy.StandardInterceptor.Intercept(IInvocation invocation)
   at Castle.DynamicProxy.AbstractInvocation.Proceed()
   at Castle.Proxies.IFoo2Proxy.M(String arg)
   at Submission#11.<<Initialize>>d__0.MoveNext()
--- End of stack trace from previous location ---
   at Microsoft.CodeAnalysis.Scripting.ScriptExecutionState.RunSubmissionsAsync[TResult](ImmutableArray`1 precedingExecutors, Func`2 currentExecutor, StrongBox`1 exceptionHolderOpt, Func`2 catchExceptionOpt, CancellationToken cancellationToken)